In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=370dfcfb2f6a790edc87a1b95243ab0bf237fdc19e1647f22fb950f2f5af1c74
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# metadata
print(breast_cancer_wisconsin_diagnostic.metadata)

# variable information
print(breast_cancer_wisconsin_diagnostic.variables)

{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'published_in': 'Electronic imaging', 'year': 1993, 'url': 'https://www.semanticscholar.org/paper/53

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

In [ ]:
import pandas as pd
# Combine horizontally
bc_df = pd.concat([X , y], axis=1)
spark_bc_df = spark.createDataFrame(bc_df)


In [ ]:
print(spark_bc_df)

DataFrame[radius1: double, texture1: double, perimeter1: double, area1: double, smoothness1: double, compactness1: double, concavity1: double, concave_points1: double, symmetry1: double, fractal_dimension1: double, radius2: double, texture2: double, perimeter2: double, area2: double, smoothness2: double, compactness2: double, concavity2: double, concave_points2: double, symmetry2: double, fractal_dimension2: double, radius3: double, texture3: double, perimeter3: double, area3: double, smoothness3: double, compactness3: double, concavity3: double, concave_points3: double, symmetry3: double, fractal_dimension3: double, Diagnosis: string]


In [ ]:
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer

from pyspark.ml.feature import VectorAssembler

X_column_names = X.columns.tolist()

#vectorAssembler=VectorAssembler(inputCols=["radius1","texture1", "perimeter1","area1"], outputCol="features")
vectorAssembler=VectorAssembler(inputCols=X_column_names, outputCol="features")
vbc_df=vectorAssembler.transform(spark_bc_df)

indexer=StringIndexer(inputCol="Diagnosis", outputCol="label")
ivbc_df=indexer.fit(vbc_df).transform(vbc_df)
ivbc_df.show(1)






+-------+--------+----------+------+-----------+------------+----------+---------------+---------+------------------+-------+--------+----------+-----+-----------+------------+----------+---------------+---------+------------------+-------+--------+----------+------+-----------+------------+----------+---------------+---------+------------------+---------+--------------------+-----+
|radius1|texture1|perimeter1| area1|smoothness1|compactness1|concavity1|concave_points1|symmetry1|fractal_dimension1|radius2|texture2|perimeter2|area2|smoothness2|compactness2|concavity2|concave_points2|symmetry2|fractal_dimension2|radius3|texture3|perimeter3| area3|smoothness3|compactness3|concavity3|concave_points3|symmetry3|fractal_dimension3|Diagnosis|            features|label|
+-------+--------+----------+------+-----------+------------+----------+---------------+---------+------------------+-------+--------+----------+-----+-----------+------------+----------+---------------+---------+---------------

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt=DecisionTreeClassifier(featuresCol="features",labelCol="label")

splits=ivbc_df.randomSplit([0.6,0.4],1)
train_df=splits[0]
test_df=splits[1]


dt_model=dt.fit(train_df)
dt_predictions=dt_model.transform(test_df)

dt_evaluator=MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction", metricName="accuracy")
dt_accuracy=dt_evaluator.evaluate(dt_predictions)
dt_accuracy

0.8468468468468469

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
layers=[30,5,5,2]
mlp=MultilayerPerceptronClassifier(layers=layers, seed=1)

splits=ivbc_df.randomSplit([0.6,0.4],1)
train_df=splits[0]
test_df=splits[1]


mlp_model=mlp.fit(train_df)
mlp_predictions=mlp_model.transform(test_df)

mlp_evaluator=MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction", metricName="accuracy")
mlp_accuracy=mlp_evaluator.evaluate(dt_predictions)
mlp_accuracy

0.8468468468468469